
# Task 1 – Daten-Preprocessing

Ziel: Rohdaten aus `buildings.csv` explorieren, bereinigen, normalisieren und für weitere Schritte (Anreicherung/Visualisierung) vorbereiten.

Hinweis: Bei der Verwendung der [Wikimedia PAWS Instanz](https://hub-paws.wmcloud.org/hub) bitte dieses Notebook und die zugehörigen Dateien separat hochladen (Upload-Symbol in der Seitenleiste). Das Projekt findet sich im [GitHub-Repository](https://github.com/kristbaum/historical-data-hacking).

## Installation von Abhängigkeiten

* [Pandas](https://pandas.pydata.org/) – Datenanalyse-Toolkit für tabellarische Daten (DataFrames) zum Einlesen, Filtern und Transformieren.
* [NumPy](https://numpy.org/) – Numerische Grundbibliothek für effiziente Arrays sowie Vektor- und Matrixrechnungen.
* [Shapely](https://shapely.readthedocs.io/en/stable/) – Geometrieoperationen (Points, Polygons) und räumliche Analysen, z. B. Punkt-in-Polygon.

In [ ]:
# Install required packages into this notebook's Python kernel
%pip install -q --upgrade pip
%pip install -q pandas numpy shapely  # shapely for geospatial stuff

In [ ]:
import pandas as pd
import numpy as np

# Viele Spalten enthalten gemischte / freie Werte
# → zunächst als Strings importieren vermeidez fehlerhafte automatische Typkonvertierungen.
buildings = pd.read_csv('buildings.csv', dtype=str)
print(buildings.shape)

# Zeigt einen Ausschnitt der ersten 5 Zeilen der Tabelle
buildings.head()


## 1. Erste Profilierung

1. Zähle Null-/Leerwerte pro Spalte (leerer String `''` zählt als leer).
2. Ermittle den Anteil gefüllter Werte je Spalte.
3. Erzeuge ein [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) mit: Spaltenname, Non-Null-%, Anzahl unterschiedlicher Werte.
4. Entferne leere bzw. nahezu leere Spalten, um das Datenset übersichtlicher zu machen.

In [ ]:
# 1.
# Ersetze Leerstrings und Spaces durch NA (fehlende Werte)
data_with_normalized_nulls = buildings.replace('', np.nan)

# Liste für Spaltenprofile erstellen
profile_list = []

# Iteriere über alle Spalten und sammle Statistiken
for column_name in data_with_normalized_nulls.columns:
    column_series = data_with_normalized_nulls[column_name]
    
    # Berechne Anteil nicht-Null Werte (in Prozent)
    non_null_percentage = column_series.notna().mean() * 100
    
    # Zähle einzigartige Werte (ohne NA)
    unique_value_count = column_series.nunique(dropna=True)
    
    
    # Füge Profil zur Liste hinzu
    profile_list.append({
        'column': column_name,
        'non_null_pct': non_null_percentage,
        'n_unique': unique_value_count,
    })

# Erstelle DataFrame aus den Profilen und sortiere nach Anteil nicht-Null Werte
print(pd.DataFrame(profile_list))



In [ ]:
# 4. Entferne Spalten mit sehr wenigen/nicht vorhandenen Werten
# Schwelle in Prozent (anpassbar): Spalten mit weniger als diesem Anteil an nicht-leeren Werten werden entfernt
min_non_null_pct = 5.0  # z. B. 5%

# Behandle '' und ' ' als fehlend für die Berechnung (ohne Side-Effects auf buildings)
normalized = buildings.replace({'': pd.NA, ' ': pd.NA}).infer_objects(copy=False)
non_null_pct = normalized.notna().mean() * 100

# Optional: wichtige Spalten schützen, damit sie nicht versehentlich entfernt werden
protected_cols = [c for c in ['ID', 'appellation', 'verbaleDating', 'locationLat', 'locationLng'] if c in buildings.columns]

# Bestimme zu entfernende Spalten
_to_drop = non_null_pct[non_null_pct < min_non_null_pct].index.tolist()
# geschützte Spalten nicht droppen
_to_drop = [c for c in _to_drop if c not in protected_cols]

# Entferne Spalten in-place
before_cols = buildings.shape[1]
buildings.drop(columns=_to_drop, inplace=True, errors='ignore')
after_cols = buildings.shape[1]

print(f"Entfernte Spalten (non-null < {min_non_null_pct}%): {len(_to_drop)} | Vorher: {before_cols}, Nachher: {after_cols}")


# Optional: zeige die 10 leersten verbleibenden Spalten zur Kontrolle
print('\nLeerste verbleibende Spalten (Top 10):')
print(non_null_pct.loc[buildings.columns].sort_values().head(10))

Fragen:

- Welche Spalten sind fast leer und sollten ggf. ausgelagert oder ignoriert werden?
- Gibt es offensichtliche Duplikate bei `ID`?

## 2. Bereinigung `verbaleDating`

Die Spalte enthält verbale Zeitangaben/Bereiche, z. B.: `"1000-2020, 1773-1776"`.

Aufgaben:

1. Zerlege `verbaleDating` in einzelne Segmente (Trennzeichen: Komma) und normalisiere Leerzeichen.
2. Identifiziere Bereiche (Pattern `YYYY-YYYY`) vs. Einzeljahre (`YYYY`).
3. Baue eine normalisierte Tabelle `building_dates`: (`building_id`, `date_raw`, `year_start`, `year_end`, `is_range`, `precision`).
4. Berechne je Gebäude: minimaler Start und maximaler Endwert → `chronology_min`, `chronology_max`, und füge sie dem Haupt-DataFrame wieder hinzu.
5. Detektiere Ausreißer (z. B. Jahr < 1000 oder > aktuelles Jahr) und markiere sie für manuelle Prüfung.

In [ ]:
import re

# - Wir suchen die ersten 1–2 vierstelligen Jahreszahlen (\bYYYY\b) in der
#   verbal_Dating SPalte in der Reihenfolge ihres Auftretens.
# - Wenn nur ein Jahr gefunden wird, setzen wir year_start = year_end = dieses Jahr.
# - Wenn zwei Jahre gefunden werden, verwenden wir das erste als year_start und das
#   zweite als year_end.

def _parse_first_two_years(s):
    if not isinstance(s, str):
        return (pd.NA, pd.NA)
    matches = re.findall(r"\b(\d{4})\b", s)
    if not matches:
        return (pd.NA, pd.NA)
    y1 = int(matches[0])
    y2 = int(matches[1]) if len(matches) > 1 else y1
    return (y1, y2)

# Wende parser auf jede Zeile an
years = buildings['verbaleDating'].fillna('').map(_parse_first_two_years)
# Zerlege die Tupel in eine DataFrame mit gleichen Indices
year_df = pd.DataFrame(years.tolist(), index=buildings.index, columns=['year_start','year_end'])

# Schreibe zurück in 'buildings' als Zahl
buildings['year_start'] = year_df['year_start'].astype('Int64')
buildings['year_end']   = year_df['year_end'].astype('Int64')

# Einfache Kennzahlen und Beispiele
vd_raw = buildings['verbaleDating'].fillna('')
rows_with_text = int(vd_raw.str.strip().ne('').sum())
rows_with_parsed = int(buildings[['year_start','year_end']].notna().any(axis=1).sum())

print(f"Nicht leere verbal_Dating Zeilen: {rows_with_text}")
print(f"Zeilen mit zugeordneten Jahr(en): {rows_with_parsed} von {len(buildings)}")

# Beispiele: Vorher -> Nachher (year_start/year_end)
examples = buildings.loc[vd_raw.str.strip().ne(''), ['verbaleDating','year_start','year_end']].head(10)
print("\nBeispiele (Vorher: verbaleDating → Nachher: year_start/year_end):")
print(examples)


## 3. Geodaten-Auswertung

1. Konvertiere `locationLat`/`locationLng` zu Fließkommazahlen.
2. Prüfe Wertebereiche der Koordinaten (alle sollten ca. in DE liegen; [Eckpunkte siehe Wikidata](https://www.wikidata.org/wiki/Q183)).
3. Finde alle Gebäude im heutigen Hessen per Bounding Box (gedachtes Rechteck; grob von einer Karte geschätzt).
4. Beschränke auf die historische [Provinz Hessen-Nassau](https://de.wikipedia.org/wiki/Provinz_Hessen-Nassau) via Geoshape. Daten von [OpenHistoricalMap](https://www.openhistoricalmap.org/relation/2690442), Abfrage via [Overpass](https://overpass-turbo.openhistoricalmap.org/?Q=%5Bout%3Ajson%5D%5Btimeout%3A120%5D%3B%0Arel%282690442%29%3B%0A%28._%3B%3E%3B%29%3B%0Aout%20body%20geom%3B&C=50.004209%3B11.518449%3B7) (Export → Geoshape).

In [ ]:
# 1. Konvertierung zu Fließkommazahlen
def to_float(s):
    try:
        return float(s)
    except (TypeError, ValueError):
        return pd.NA

coords = buildings[['locationLat','locationLng']].map(to_float)

In [ ]:
# 2. Wertebereich in ca. DE
valid_lat = coords.locationLat.between(47,55)
valid_lng = coords.locationLng.between(5,16)
buildings['coord_valid'] = valid_lat & valid_lng


# Liste invalide Locations
invalid_mask = ~buildings['coord_valid'].fillna(False)
if invalid_mask.any():
    print(buildings.loc[invalid_mask, ['ID','appellation','locationLat','locationLng']])
else:
    print("Keine ungültigen Koordinaten gefunden.")


In [ ]:
# 3 Bounding Box für Hessen
# Bounding Box ungefähr für Hessen (Nordwest / Südost):
NW_LAT, NW_LNG = 51.599, 8.072   # Nordwestlichster Punkt
SE_LAT, SE_LNG = 49.495, 10.088  # Südöstlichster Punkt


# Min/Max für Bereichsprüfung bestimmen
min_lat, max_lat = SE_LAT, NW_LAT
min_lng, max_lng = NW_LNG, SE_LNG

# Masken für BBox prüfen (nur valide Koordinaten berücksichtigen)
in_bbox = (
    coords.locationLat.between(min_lat, max_lat)
    & coords.locationLng.between(min_lng, max_lng)
)
mask_bbox = in_bbox & buildings['coord_valid'].fillna(False)

# Ergebnis: Gefilterte Entitäten in Hessen (via BBox)
buildings['in_hesse_bbox'] = mask_bbox
buildings_hesse_bbox = buildings.loc[mask_bbox].copy()

print(
    f"BBox Hessen: lat [{min_lat}, {max_lat}], lng [{min_lng}, {max_lng}] | "
    f"Treffer: {mask_bbox.sum()} von {len(buildings)}"
)
buildings_hesse_bbox.head(10)

# Wie man an addressState ganz gut erkennen kann, sind auch ein paar Falsche mit dabei

In [ ]:
# 4 Punkt-in-Polygon: Historisches Hessen-Nassau
import json
from shapely.geometry import shape, Point

# GeoJSON laden (Datei im aktuellen Ordner erwartet)
with open('./hessen_nassau.geojson', 'r', encoding='utf-8') as f:
    gj = json.load(f)

# Geometrie aus FeatureCollection/Feature/Geometry entnehmen
geom = (
    gj['features'][0]['geometry'] if gj.get('type') == 'FeatureCollection'
    else (gj['geometry'] if gj.get('type') == 'Feature' else gj)
)
hesse_nassau_polygon = shape(geom)

# Nur auf Zeilen mit gültigen Koordinaten prüfen
valid_mask = buildings['coord_valid'].fillna(False)
inside_series = pd.Series(False, index=buildings.index)
inside_series.loc[valid_mask] = coords.loc[valid_mask].apply(
    lambda r: hesse_nassau_polygon.contains(Point(float(r['locationLng']), float(r['locationLat']))),
    axis=1
)

buildings['in_hesse_nassau_geo'] = inside_series
print(f"Geo-Shape (Hessen-Nassau): Treffer {buildings['in_hesse_nassau_geo'].sum()} von {len(buildings)}")
buildings.loc[buildings['in_hesse_nassau_geo']].head(10)

In [ ]:
# Export current 'buildings' DataFrame for OpenRefine
out_path = './buildings_openrefine.csv'
buildings.to_csv(out_path, index=False, encoding='utf-8')
print(f"Exported buildings -> {out_path} ({buildings.shape[0]} rows, {buildings.shape[1]} cols)")

## OpenRefine

Bereinigung und Normalisierung zentraler Felder aus `buildings_openrefine.csv` mit OpenRefine. Viele der Pandas-Schritte wären in OpenRefine sehr aufwändig; daher arbeiten wir auf dem Export der Vorverarbeitung weiter.

### 1. Import `buildings_openrefine.csv`

- Lade `buildings_openrefine.csv` in OpenRefine (Create Project → File from this Computer).
- Belasse alle Spalten zunächst als Text (Option „Attempt to parse cell text into numbers“ deaktiviert lassen).

### 2. Grundlegende Bereinigungen

- Spalte `appellation`: Edit cells → Common transforms → Trim leading and trailing whitespace und ggf. Collapse consecutive whitespace.
- Erzeuge eine Duplikat-Facette auf `appellation` (Facets → Customized facets → Facet by duplicates). Markiere doppelte Zeilen per Flag, filtere unter All → Facet by flag → true und entferne sie über All → Edit rows → Remove matching rows.

### 4. `people.csv` bereinigen

- `people.csv` ins Projekt laden.
- Überflüssige Spalten entfernen:
  * `verbaleDating` ist leer: prüfen über Spaltenmenü → Facet → Text facet; anschließend löschen via Edit column → Remove this column.
  * Gleiches mit der Spalte `resource_urls` durchführen.

### 5. Clustering zur Dublettenerkennung

[Clustering](https://openrefine.org/docs/technical-reference/clustering-in-depth) ist ein fortgeschrittener Ansatz zur Erkennung von Dubletten (Tippfehler, alternative Schreibweisen).

- Spalte `appellation`: Edit cells → Cluster and edit.
- Im Clustering-Dialog zuerst Methode „Key collision“ mit Keying function „Fingerprint“ ausführen (Cluster). Diese Methode findet u. a. Wortdreher/falsch positionierte Buchstaben. Prüfen, ob echte Duplikate vorliegen.
- Danach Methode „Nearest neighbour“ mit Distanzfunktion „Levenshtein“ (erneut Cluster). Größerer Radius findet mehr Unterschiede, produziert aber auch mehr False Positives. In beiden Fällen gibt es keine echten Duplikate
